Идея: попытаться применить дерево решений так, чтобы в каждом листе было ~ 100 элементов, и для каждого листа затем использовать линейную регрессию

замечания по данным в целом такие же как и в решении линейной регрессией: удалять опасные фичи, но искать регрессию мы будем через точный аналитический метод

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import RegressorMixin
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

In [3]:
class TreeLinearRegressor(RegressorMixin):
    def __init__(self, *args, **kwargs):
        self.my_tree = DecisionTreeRegressor(*args, **kwargs)
        self.linear_regressors = {}
        self.leaf_values = {}
    def fit(self, X, y):
        self.my_tree.fit(X, y)
        leaf_indices = self.my_tree.apply(X)
        X = np.array(X)
        y = np.array(y)
        for leaf in np.unique(leaf_indices):
            indices = np.where(leaf_indices == leaf)[0]
            Xl = X[indices]
            yl = y[indices]
            self.linear_regressors[leaf] = LinearRegression()
            self.linear_regressors[leaf].fit(Xl, yl)
            self.leaf_values[leaf] = yl.mean()
        return self
    def get_params(self, **kwargs):
        return self.my_tree.get_params(**kwargs)
    def set_params(self, *args, **kwargs):
        self.my_tree.set_params(*args, **kwargs)
    def predict(self, X):
        leaf_indices = self.my_tree.apply(X)
        predictions = np.zeros(X.shape[0])

        for i, x in enumerate(X):
            leaf_index = self.my_tree.apply([x])[0]  # Находим, в какой лист попадает пример
            predictions[i] = self.linear_regressors[leaf_index].predict([x])[0]
            if predictions[i] == np.nan:
                predictions[i] = self.leaf_values[leaf_index]
        return predictions

In [25]:
data = pd.read_csv('data/data.csv')
trash = ['PID', 'Order', 'Total Bsmt SF', 'Gr Liv Area', 'TotRms AbvGrd', 'SalePrice']
continuous_columns = [key for key in data.keys() if data[key].dtype in ('int64', 'float64')]
continuous_columns = list(set(continuous_columns).difference(set(trash)))   
chosen_data = ['Lot Area', 'Lot Frontage', 'Overall Cond','Overall Qual',
               '1st Flr SF', '2nd Flr SF']
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
imp = SimpleImputer()
imp.fit(data[continuous_columns])
xtr, xt, ytr, yt = train_test_split(imp.transform(data[continuous_columns]), np.array(data['SalePrice']))

In [34]:
m = TreeLinearRegressor(max_depth=7, min_samples_leaf=100)
m.fit(xtr, ytr)

In [35]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(yt, m.predict(xt))

19175.92259460624

In [11]:
def make_pipeline(chosen_data):
    from sklearn.impute import SimpleImputer
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    m = DecisionTreeRegressor()
    imp = SimpleImputer()
    scaler = StandardScaler()
    cols = ColumnTransformer([
        ('imputer', imp, chosen_data),
    ])
    pipe = Pipeline([
        ('cols_transformer', cols),
        ('tree-lin-reg', m)
    ])
    return pipe


In [16]:
from sklearn.model_selection import GridSearchCV
data = pd.read_csv('data/data.csv')
data = data[data['SalePrice'] < 300000]
data = data[data['SalePrice'] > 50000]
trash = ['PID', 'Order', 'Total Bsmt SF', 'Gr Liv Area', 'TotRms AbvGrd', 'SalePrice']
continuous_columns = [key for key in data.keys() if data[key].dtype in ('int64', 'float64')]
continuous_columns = list(set(continuous_columns).difference(set(trash)))   
params={
    'tree-lin-reg__max_depth': list(range(3, 14, 3)),
    'tree-lin-reg__min_samples_leaf': [i for i in range(10, 71, 10)],
    'tree-lin-reg__criterion': ['absolute_error', 'poisson', 'squared_error'],
    'tree-lin-reg__splitter': ['best', 'random'],
}
chosen_data = ['Lot Area', 'Lot Frontage', 'Overall Cond','Overall Qual',
               '1st Flr SF', '2nd Flr SF']

grids = GridSearchCV(
    estimator=make_pipeline(continuous_columns),
    param_grid=params,
    cv=4,
    scoring='neg_mean_absolute_error',
)

grids.fit(data[continuous_columns], data['SalePrice'])

C:\python311\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('cols_transformer',
                                        ColumnTransformer(transformers=[('imputer',
                                                                         SimpleImputer(),
                                                                         ['Garage '
                                                                          'Area',
                                                                          'BsmtFin '
                                                                          'SF '
                                                                          '2',
                                                                          'BsmtFin '
                                                                          'SF '
                                                                          '1',
                                                                          'Low '
                                                                          'Qual '
                                                                          'Fin '
                                                                          'SF',
                                                                          'Lot '
                                                                          'Area',
                                                                          'Mas '
                                                                          'Vnr '
                                                                          'Area',
                                                                          'Bsmt '
                                                                          'Unf '
                                                                          'SF',
                                                                          'Overall '
                                                                          'Qual',
                                                                          'Overall '
                                                                          'Cond',
                                                                          '2nd '
                                                                          'Flr '
                                                                          'SF',
                                                                          'Wood '
                                                                          'Deck '
                                                                          'SF',
                                                                          '3Ssn '
                                                                          'Porch',
                                                                          'Enclosed '
                                                                          'Porch',
                                                                          'Half '
                                                                          'Bath',
                                                                          '1...
                                                                          'Blt',
                                                                          'Full '
                                                                          'Bath',
                                                                          'Kitchen '
                                                                          'AbvGr',
                                                                          'Mo '
                                                                          'Sold', ...])])),
                                       ('tree-lin-reg',
                                        DecisionTreeRegressor())]),
            

In [17]:
grids.best_score_

-19051.50223880597

In [14]:
grids.best_params_

{'tree-lin-reg__max_depth': 11, 'tree-lin-reg__min_samples_leaf': 30}

В общем, после проб разных параметров я пришёл к выводу, что дерево решений в целом плохо описывает данные.
В ходе реализации идеи получалось так, что TreeLinearRegression становилась лучше только тогда, когда минимальное количество элементов на лист дерева увеличивалось --> модель вырождалась в линейную регрессию.

Остается применять ансамблирование.